In [93]:
import matplotlib

%matplotlib inline

In [94]:
# The first thing to do is to import the relevant packages
# that I will need for my script, 
# these include the Numpy (for maths and arrays)
# and csv for reading and writing csv files
# If i want to use something from this I need to call 
# csv.[function] or np.[function] first

import csv as csv 
import numpy as np

# Open up the csv file in to a Python object
data_all = []
with open('events.csv', encoding="utf8") as train_file:
    csv_reader = csv.reader(train_file, delimiter=';', quotechar='"')
    for row in csv_reader:
        data_all.append(row)
data_all = np.array(data_all)
data = data_all[1::]


In [95]:
import pandas as pd
import re

data = pd.read_csv('events.csv', sep=';')

#population_age = df.Link.astype(np.float)
#print(data.Description)
#print(data.Details)

df = data[['Description','Details']]

df[0:10]


,Description,Details
0,"Rencontre autour des nouveaux usages, de la pa...",Que recouvre aujourd’hui la notion de particip...
1,Découverte des risques toxicologiques des cham...,Découvrez les risques toxicologiques par des d...
2,Une enquête sur un empoisonnement,Devenez détectives et élucidez l'enquête qui v...
3,Apprendre à dépister des substances toxiques,"En laboratoire, comme dans la vie de tous les ..."
4,Expositions et animations autour de l'expérime...,**Le Réseau des maisons de l’architecture orga...
5,Agence de marketing d'influence,NaN
6,dans le cadre du festival Bar-Bars,Déroutant et imprévisible : Gratuit ne fait ja...
7,dans le cadre du festival Bar-Bars,Dj Barbarpapa en exclusivité au Barablabla le ...
8,dans le cadre du festival Bar-Bars,Nouch'Ma est un duo guitare/violon naviguant e...
9,dans le cadre du festival Bar-Bars,"Bas d'Plafond, c'est quelque part entre Babylo..."


## Netoyage des données

LDA a besoin que chaque phrase soit découpé sous forme d'un tableau de mots minuscules. On réfléchira par la suite sur le fait que plusieurs

In [96]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')


for item in data.Description:
    if type(item) is str :
        raw = item.lower()
        token = tokenizer.tokenize(raw)
        #print(token)




In [129]:
from stop_words import get_stop_words

# create English stop words list
#en_stop = get_stop_words('en')

# create French stop words list
fr_stop = get_stop_words('fr')
#fr_stop.append('nan')
#fr_stop.append('s')
#fr_stop.append('c')
#fr_stop.append('http')
#fr_stop.append('www')
#fr_stop.append('nantes')
#fr_stop.append('nantais')
#fr_stop.append('nantaise')
#fr_stop.append("com")
#fr_stop.append("youtube")
#fr_stop.append("facebook")
#fr_stop.append('https')
#fr_stop.append("fr")
#fr_stop.append("lundi")
#fr_stop.append("mardi")
#fr_stop.append("mercredi")
#fr_stop.append("jeudi")
#fr_stop.append("vendredi")
#fr_stop.append("samedi")
#fr_stop.append("dimanche")
#fr_stop.append("be");
#fr_stop.append("v");

print(fr_stop)


['a', 'ai', 'aie', 'aient', 'aies', 'ait', 'alors', 'as', 'au', 'aucun', 'aura', 'aurai', 'auraient', 'aurais', 'aurait', 'auras', 'aurez', 'auriez', 'aurions', 'aurons', 'auront', 'aussi', 'autre', 'aux', 'avaient', 'avais', 'avait', 'avant', 'avec', 'avez', 'aviez', 'avions', 'avoir', 'avons', 'ayant', 'ayez', 'ayons', 'bon', 'car', 'ce', 'ceci', 'cela', 'ces', 'cet', 'cette', 'ceux', 'chaque', 'ci', 'comme', 'comment', 'd', 'dans', 'de', 'dedans', 'dehors', 'depuis', 'des', 'deux', 'devoir', 'devrait', 'devrez', 'devriez', 'devrions', 'devrons', 'devront', 'dois', 'doit', 'donc', 'dos', 'droite', 'du', 'dès', 'début', 'dù', 'elle', 'elles', 'en', 'encore', 'es', 'est', 'et', 'eu', 'eue', 'eues', 'eurent', 'eus', 'eusse', 'eussent', 'eusses', 'eussiez', 'eussions', 'eut', 'eux', 'eûmes', 'eût', 'eûtes', 'faire', 'fais', 'faisez', 'fait', 'faites', 'fois', 'font', 'force', 'furent', 'fus', 'fusse', 'fussent', 'fusses', 'fussiez', 'fussions', 'fut', 'fûmes', 'fût', 'fûtes', 'haut', 'ho

# Stop words
Certain parts of English speech, like conjunctions (“for”, “or”) or the word “the” are meaningless to a topic model. 
These terms are called stop words and need to be removed from our token list.

#### On enlève les STOP WORDS des phrases

In [120]:

tokens_stop_serie = pd.Series()
i=0

for item in data.Description:
    if item != "dans le cadre du festival Bar-Bars":
        raw = str(item) +' '+ str(data.Details[i])
        token = tokenizer.tokenize(raw.lower())
        stopped_tokens = [i for i in token if not i in fr_stop]
        tokens_stop_serie = tokens_stop_serie.set_value(i,stopped_tokens)
        #print(tokens_stop_serie[i])
        i+=1

        
    


# Stemming
Stemming words is another common NLP technique to reduce topically similar words to their root. 
For example, “nantaise,” “nantes,” “nantais,” all have similar meanings; stemming reduces those terms to “nant” 
This is important for topic modeling, which would otherwise view those terms as separate entities and reduce 
their importance in the model

In [121]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import FrenchStemmer

stemmer = FrenchStemmer()
tokens_stem_serie = pd.Series()
i = 0
# Create p_stemmer of class PorterStemmer
# p_stemmer = PorterStemmer()

for stop_word_token in tokens_stop_serie:
    # stem token
    stemmed_tokens = [stemmer.stem(i) for i in stop_word_token]
    tokens_stem_serie = tokens_stem_serie.set_value(i,stemmed_tokens)
    i += 1



# Constructing a document-term matrix

In [122]:
from gensim import corpora, models

dictionary = corpora.Dictionary(tokens_stem_serie)

corpus = [dictionary.doc2bow(token) for token in tokens_stem_serie]

# The tuples are (term ID, term frequency)
print(corpus[0])





INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(6604 unique tokens: ['vot', 'racism', 'init', 'mosaïqu', 'madein']...) from 642 documents (total 27692 corpus positions)


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 3), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 5), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 2), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1)]


## Applying the LDA model
corpus is a document-term matrix and now we’re ready to generate an LDA model:

Parameters:

num_topics: required. An LDA model requires the user to determine how many topics should be generated. Our document set is small, so we’re only asking for three topics.
id2word: required. The LdaModel class requires our previous dictionary to map ids to strings.
passes: optional. The number of laps the model will take through corpus. The greater the number of passes, the more accurate the model will be. A lot of passes can be slow on a very large corpus.

In [130]:

ldamodel = models.ldamodel.LdaModel(corpus, num_topics=15, id2word = dictionary, passes=50)
                                    

INFO:gensim.models.ldamodel:using symmetric alpha at 0.06666666666666667
INFO:gensim.models.ldamodel:using symmetric eta at 0.00015142337976983646
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online LDA training, 15 topics, 50 passes over the supplied corpus of 642 documents, updating model once every 642 documents, evaluating perplexity every 642 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:-14.582 per-word bound, 24533.4 perplexity estimate based on a held-out corpus of 642 documents with 27692 words
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #642/642
INFO:gensim.models.ldamodel:topic #5 (0.067): 0.006*"visit" + 0.006*"musiqu" + 0.005*"rock" + 0.004*"présent" + 0.004*"anim" + 0.004*"group" + 0.003*"guid" + 0.003*"voix" + 0.003*"exposit" + 0.003*"part"
INFO:gensim.models.ldamodel:topic #12 (0.067): 0.006*"numer" + 0.005*"création" + 0.005*"rend" + 0.004*"amp" + 0.00

In [131]:
print(ldamodel.print_topics(num_topics=15, num_words=3))

INFO:gensim.models.ldamodel:topic #0 (0.067): 0.009*"westcostar" + 0.006*"musiqu" + 0.006*"découvr"
INFO:gensim.models.ldamodel:topic #1 (0.067): 0.015*"utilis" + 0.014*"contenu" + 0.014*"contact"
INFO:gensim.models.ldamodel:topic #2 (0.067): 0.008*"projet" + 0.007*"jou" + 0.007*"openstreetmap"
INFO:gensim.models.ldamodel:topic #3 (0.067): 0.007*"dj" + 0.007*"développ" + 0.007*"univers"
INFO:gensim.models.ldamodel:topic #4 (0.067): 0.010*"jsp" + 0.007*"amp" + 0.007*"rend"
INFO:gensim.models.ldamodel:topic #5 (0.067): 0.008*"musiqu" + 0.006*"scen" + 0.006*"rock"
INFO:gensim.models.ldamodel:topic #6 (0.067): 0.005*"blog" + 0.005*"histoir" + 0.005*"présent"
INFO:gensim.models.ldamodel:topic #7 (0.067): 0.010*"blu" + 0.007*"associ" + 0.007*"rock"
INFO:gensim.models.ldamodel:topic #8 (0.067): 0.006*"plus" + 0.005*"be" + 0.005*"youtu"
INFO:gensim.models.ldamodel:topic #9 (0.067): 0.007*"lieux" + 0.006*"co" + 0.006*"tier"
INFO:gensim.models.ldamodel:topic #10 (0.067): 0.009*"the" + 0.008*"hip

[(0, '0.009*"westcostar" + 0.006*"musiqu" + 0.006*"découvr"'), (1, '0.015*"utilis" + 0.014*"contenu" + 0.014*"contact"'), (2, '0.008*"projet" + 0.007*"jou" + 0.007*"openstreetmap"'), (3, '0.007*"dj" + 0.007*"développ" + 0.007*"univers"'), (4, '0.010*"jsp" + 0.007*"amp" + 0.007*"rend"'), (5, '0.008*"musiqu" + 0.006*"scen" + 0.006*"rock"'), (6, '0.005*"blog" + 0.005*"histoir" + 0.005*"présent"'), (7, '0.010*"blu" + 0.007*"associ" + 0.007*"rock"'), (8, '0.006*"plus" + 0.005*"be" + 0.005*"youtu"'), (9, '0.007*"lieux" + 0.006*"co" + 0.006*"tier"'), (10, '0.009*"the" + 0.008*"hip" + 0.008*"hop"'), (11, '0.009*"musiqu" + 0.008*"don" + 0.008*"photos"'), (12, '0.008*"numer" + 0.007*"jsp" + 0.006*"amp"'), (13, '0.007*"musiqu" + 0.006*"rock" + 0.005*"anné"'), (14, '0.028*"visit" + 0.015*"exposit" + 0.013*"guid"')]


In [103]:
import numpy as np
import lda
import lda.datasets

X = lda.datasets.load_reuters()
vocab = lda.datasets.load_reuters_vocab()
titles = lda.datasets.load_reuters_titles()
X.shape
(395, 4258)
X.sum()
84010
model = lda.LDA(n_topics=20, n_iter=1500, random_state=1)
model.fit(X)  # model.fit_transform(X) is also available
topic_word = model.topic_word_  # model.components_ also works
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]     
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))
    



INFO:lda:n_documents: 395
INFO:lda:vocab_size: 4258
INFO:lda:n_words: 84010
INFO:lda:n_topics: 20
INFO:lda:n_iter: 1500
INFO:lda:<0> log likelihood: -1051748
INFO:lda:<10> log likelihood: -719800
INFO:lda:<20> log likelihood: -699115
INFO:lda:<30> log likelihood: -689370
INFO:lda:<40> log likelihood: -684918
INFO:lda:<50> log likelihood: -681322
INFO:lda:<60> log likelihood: -678979
INFO:lda:<70> log likelihood: -676598
INFO:lda:<80> log likelihood: -675383
INFO:lda:<90> log likelihood: -673316
INFO:lda:<100> log likelihood: -672761
INFO:lda:<110> log likelihood: -671320
INFO:lda:<120> log likelihood: -669744
INFO:lda:<130> log likelihood: -669292
INFO:lda:<140> log likelihood: -667940
INFO:lda:<150> log likelihood: -668038
INFO:lda:<160> log likelihood: -667429
INFO:lda:<170> log likelihood: -666475
INFO:lda:<180> log likelihood: -665562
INFO:lda:<190> log likelihood: -664920
INFO:lda:<200> log likelihood: -664979
INFO:lda:<210> log likelihood: -664722
INFO:lda:<220> log likelihood: -

Topic 0: british churchill sale million major letters west britain
Topic 1: church government political country state people party against
Topic 2: elvis king fans presley life concert young death
Topic 3: yeltsin russian russia president kremlin moscow michael operation
Topic 4: pope vatican paul john surgery hospital pontiff rome
Topic 5: family funeral police miami versace cunanan city service
Topic 6: simpson former years court president wife south church
Topic 7: order mother successor election nuns church nirmala head
Topic 8: charles prince diana royal king queen parker bowles
Topic 9: film french france against bardot paris poster animal
Topic 10: germany german war nazi letter christian book jews
Topic 11: east peace prize award timor quebec belo leader
Topic 12: n't life show told very love television father
Topic 13: years year time last church world people say
Topic 14: mother teresa heart calcutta charity nun hospital missionaries
Topic 15: city salonika capital buddhist c

In [104]:
doc_topic = model.doc_topic_
for i in range(10):
    print("{} (top topic: {})".format(titles[i], doc_topic[i].argmax()))

0 UK: Prince Charles spearheads British royal revolution. LONDON 1996-08-20 (top topic: 8)
1 GERMANY: Historic Dresden church rising from WW2 ashes. DRESDEN, Germany 1996-08-21 (top topic: 13)
2 INDIA: Mother Teresa's condition said still unstable. CALCUTTA 1996-08-23 (top topic: 14)
3 UK: Palace warns British weekly over Charles pictures. LONDON 1996-08-25 (top topic: 8)
4 INDIA: Mother Teresa, slightly stronger, blesses nuns. CALCUTTA 1996-08-25 (top topic: 14)
5 INDIA: Mother Teresa's condition unchanged, thousands pray. CALCUTTA 1996-08-25 (top topic: 14)
6 INDIA: Mother Teresa shows signs of strength, blesses nuns. CALCUTTA 1996-08-26 (top topic: 14)
7 INDIA: Mother Teresa's condition improves, many pray. CALCUTTA, India 1996-08-25 (top topic: 14)
8 INDIA: Mother Teresa improves, nuns pray for "miracle". CALCUTTA 1996-08-26 (top topic: 14)
9 UK: Charles under fire over prospect of Queen Camilla. LONDON 1996-08-26 (top topic: 8)
